In [1]:
import pandas as pd
import numpy as np

# DataFrame和Series的区别

1. 二者很多功能是重叠的


2. 可以理解series为1维结构的, 当然也不完全是1维, 毕竟series还是有index的, Dataframe为二维结构, 其视觉参考就是Excel表, 具备行列


3. 二者的差异参见构造的支持参数

## Series

In [3]:
series = pd.Series([1, 2, 3])
series

0    1
1    2
2    3
dtype: int64

In [4]:
series_a = pd.Series(data=[1,2,3], index=['a', 'b', 'c'], name='test')
series_a

a    1
b    2
c    3
Name: test, dtype: int64

显然不能单纯认为series是类似于一维列表的数据结构

### series转dataframe

In [5]:
s_df = series.to_frame()
s_df

,0
0,1
1,2
2,3


### series转字典

In [6]:
series_a.to_dict()

{'a': 1, 'b': 2, 'c': 3}

### series编辑

In [7]:
series[[0]] = 10
series

0    10
1     2
2     3
dtype: int64

In [8]:
series.loc[0] = 20
series

0    20
1     2
2     3
dtype: int64

In [9]:
series.drop(0)

1    2
2    3
dtype: int64

注意append方法已经备逐步废弃, series, dataframe统一被归并到concat类方法, 拼接数据

In [13]:
pd.concat([pd.Series(data = [9,0]), pd.Series(data=[11,13])])

0     9
1     0
0    11
1    13
dtype: int64

## Dataframe

注意构造的参数是一个二维的数组

In [2]:
df = pd.DataFrame([[1,3,4], [25, 65, 12]])

二维数组, 字典均可作为数据源, 转换为dataframe结构

In [81]:
dic = {
    'a': [1, 2, 3],
    'b': [5, 7, 9]
}
pd.DataFrame(dic)

,a,b
0,1,5
1,2,7
2,3,9


In [3]:
df

,0,1,2
0,1,3,4
1,25,65,12


In [64]:
# 获取头部
df.head(1)

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,aa,NaN,p


变更表头名称(列名)

"*", 解包

In [65]:
# 获取尾部
df.tail(1)

,d,a,c,g,h,k,i,l,m
2,14,85,2,37,10,20,aa,cc,q


In [4]:
df.columns = [*'abc']

In [5]:
df

,a,b,c
0,1,3,4
1,25,65,12


另外的方式来变更名称

In [39]:
df.rename(columns={"a": "a_b"}, inplace=True)
df

,d,a_b,b,c,g,h,k,i
0,0,1,3,4,105,119,911,aa
1,0,25,65,12,137,110,120,cc
2,14,85,15,2,37,10,20,aa


In [40]:
df.rename(columns={"a_b": "a"}, inplace=True)
df

,d,a,b,c,g,h,k,i
0,0,1,3,4,105,119,911,aa
1,0,25,65,12,137,110,120,cc
2,14,85,15,2,37,10,20,aa


插入新的列

In [6]:
df.insert(0, 'd', 0)

In [7]:
df

,d,a,b,c
0,0,1,3,4
1,0,25,65,12


插入多列数据, assign

In [11]:
df.assign(e = lambda x: x.a + 1, f = lambda x: x.b * 2)

,d,a,b,c,e,f
0,0,1,3,4,2,6
1,0,25,65,12,26,130


In [19]:
df

,d,a,b,c
0,0,1,3,4
1,0,25,65,12


In [15]:
def test(a, c):
    return a + c

apply调用多个参数

In [20]:
df['g'] = df.apply(lambda c: test(c['a'], c['c']), axis=1)

In [21]:
df

,d,a,b,c,g
0,0,1,3,4,5
1,0,25,65,12,37


单列的数据的调用

In [23]:
df['g'] = df['g'].apply(lambda x: x + 100)

In [24]:
df

,d,a,b,c,g
0,0,1,3,4,105
1,0,25,65,12,137


当目标函数返回元组的数据时, 需要将多个数据分别放进不同的列中

In [25]:
def test_tuple(x):
    return (110, 120) if x > 2 else  (119, 911)

In [28]:
tmp = df.apply(lambda x: test_tuple(x['a']), result_type='expand', axis=1)

In [29]:
tmp

,0,1
0,119,911
1,110,120


In [30]:
tmp.columns = ['h', 'k']

In [31]:
df = pd.concat([df, tmp], axis=1)

In [32]:
df

,d,a,b,c,g,h,k
0,0,1,3,4,105,119,911
1,0,25,65,12,137,110,120


添加新的行

In [33]:
df.loc[2] = [14, 85, 15, 2, 37, 10, 20]
df

,d,a,b,c,g,h,k
0,0,1,3,4,105,119,911
1,0,25,65,12,137,110,120
2,14,85,15,2,37,10,20


In [35]:
df['i'] = ['aa', 'cc', 'aa']
df

,d,a,b,c,g,h,k,i
0,0,1,3,4,105,119,911,aa
1,0,25,65,12,137,110,120,cc
2,14,85,15,2,37,10,20,aa


读取部分的列, 同时重新排列数据

In [36]:
part_df = df[['a', 'b', 'c', 'd']]
part_df

,a,b,c,d
0,1,3,4,0
1,25,65,12,0
2,85,15,2,14


透视表

In [38]:
p_df = df.pivot_table(index='i', values=['a', 'd'], aggfunc='sum')
p_df

,a,d
i,,
aa,86,14
cc,25,0


批量执行数据求和

注意执行的合并方向, 横向, 纵向

In [41]:
df[['a', 'b', 'c']].sum(axis=1)

0      8
1    102
2    102
dtype: int64

异常值的处理

注意正无穷和负无穷是有所区别的

In [43]:
df['l'] = [np.nan, np.Inf, np.NINF]
df

,d,a,b,c,g,h,k,i,l
0,0,1,3,4,105,119,911,aa,NaN
1,0,25,65,12,137,110,120,cc,inf
2,14,85,15,2,37,10,20,aa,-inf


In [44]:
df['l'].apply(lambda x: np.isinf(x))

0    False
1     True
2     True
Name: l, dtype: bool

In [45]:
df['l'].apply(lambda x: np.isnan(x))

0     True
1    False
2    False
Name: l, dtype: bool

In [46]:
# 判断是否为负无穷
df['l'].apply(lambda x: np.isneginf(x))

0    False
1    False
2     True
Name: l, dtype: bool

In [47]:
# 判断是否为正无穷
df['l'].apply(lambda x: np.isposinf(x))

0    False
1     True
2    False
Name: l, dtype: bool

替换掉异常的无穷值

等同于 [np.inf, -np.inf]

In [48]:
df['l'].replace([np.Inf, np.NINF], ['dd', 'cc'], inplace=True)
df

,d,a,b,c,g,h,k,i,l
0,0,1,3,4,105,119,911,aa,NaN
1,0,25,65,12,137,110,120,cc,dd
2,14,85,15,2,37,10,20,aa,cc


In [49]:
# 判断数据是否为nan值
df.isna()

,d,a,b,c,g,h,k,i,l
0,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False


批量集中进行填充数据

In [63]:
df.fillna(value=999)

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,aa,999,p
1,1202,25,12,137,110,120,cc,dd,o
2,14,85,2,37,10,20,aa,cc,q


groupby

In [50]:
df.groupby(by='i')['a'].sum()

i
aa    86
cc    25
Name: a, dtype: int64

设置index

In [51]:
df.set_index('i')

,d,a,b,c,g,h,k,l
i,,,,,,,,
aa,0,1,3,4,105,119,911,NaN
cc,0,25,65,12,137,110,120,dd
aa,14,85,15,2,37,10,20,cc


In [53]:
df['m'] = [*'poq']
df

,d,a,b,c,g,h,k,i,l,m
0,0,1,3,4,105,119,911,aa,NaN,p
1,0,25,65,12,137,110,120,cc,dd,o
2,14,85,15,2,37,10,20,aa,cc,q


In [55]:
dim = df.set_index('m')
dim

,d,a,b,c,g,h,k,i,l
m,,,,,,,,,
p,0,1,3,4,105,119,911,aa,NaN
o,0,25,65,12,137,110,120,cc,dd
q,14,85,15,2,37,10,20,aa,cc


指定的列转为字典模式

In [56]:
dim['b'].to_dict()

{'p': 3, 'o': 65, 'q': 15}

删除操作

删除列

In [57]:
df.pop('b')

0     3
1    65
2    15
Name: b, dtype: int64

In [58]:
df

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,aa,NaN,p
1,0,25,12,137,110,120,cc,dd,o
2,14,85,2,37,10,20,aa,cc,q


清除掉nan值

In [59]:
df['l'].dropna()

1    dd
2    cc
Name: l, dtype: object

根据某些列进行清除整体行的操作

df.dropna(subset=['a', 'c'], inplace=True)

In [60]:
df.dropna()

,d,a,c,g,h,k,i,l,m
1,0,25,12,137,110,120,cc,dd,o
2,14,85,2,37,10,20,aa,cc,q


赋值操作

In [61]:
df.loc[1, 'd'] = 1202
df

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,aa,NaN,p
1,1202,25,12,137,110,120,cc,dd,o
2,14,85,2,37,10,20,aa,cc,q


In [68]:
# 获取列的位置
df.columns.get_loc('a')

1

获取指定的值

注意布尔值中的尽可能使用()将条件包括起来

In [69]:
df[(df['a'] > 1) & (df['h'] > 100)]

,d,a,c,g,h,k,i,l,m
1,1202,25,12,137,110,120,cc,dd,o


"~" 符号, 标识 "非"

In [70]:
df[~((df['a'] > 1) & (df['h'] > 100))]

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,aa,NaN,p
2,14,85,2,37,10,20,aa,cc,q


In [71]:
# 计算文本的长度
df['i'].str.len()

0    2
1    2
2    2
Name: i, dtype: int64

In [72]:
# 文本包含
df['i'].str.contains('a')

0     True
1    False
2     True
Name: i, dtype: bool

In [74]:
# 文本开头
df['i'].str.startswith('a')

0     True
1    False
2     True
Name: i, dtype: bool

In [75]:
# 集中转换字段的数据类型
df['k'].astype('str')

0    911
1    120
2     20
Name: k, dtype: object

In [77]:
# 返回唯一值
df['i'].nunique()

2

In [78]:
# 排序
df.sort_values(by='g')

,d,a,c,g,h,k,i,l,m
2,14,85,2,37,10,20,aa,cc,q
0,0,1,4,105,119,911,aa,NaN,p
1,1202,25,12,137,110,120,cc,dd,o


In [79]:
# 多列排序, 倒序
df.sort_values(by=['g', 'i'], ascending=False)

,d,a,c,g,h,k,i,l,m
1,1202,25,12,137,110,120,cc,dd,o
0,0,1,4,105,119,911,aa,NaN,p
2,14,85,2,37,10,20,aa,cc,q


倒置, T, 行列倒置

In [80]:
df.set_index('i').T

i,aa,cc,aa
d,0,1202,14
a,1,25,85
c,4,12,2
g,105,137,37
h,119,110,10
k,911,120,20
l,NaN,dd,cc
m,p,o,q


In [84]:
df

,d,a,c,g,h,k,i,l,m
0,0,1,4,105,119,911,bb,NaN,p
1,1202,25,12,137,110,120,cc,dd,o
2,14,85,2,37,10,20,aa,cc,q


In [95]:
data = {'i': ['aa', 'cc', 'dd'], 'z': [999, 888, 777]}
df_b = pd.DataFrame(data)
df_b

,i,z
0,aa,999
1,cc,888
2,dd,777


merge合并数据

注意这里的操作的两组数据的独一无二问题, 假如两组数据均存在重复的数据, 应当谨慎使用, 使用字典的来处理

In [96]:
df.merge(df_b, how='left', on='i')

,d,a,c,g,h,k,i,l,m,z
0,0,1,4,105,119,911,bb,NaN,p,NaN
1,1202,25,12,137,110,120,cc,dd,o,888.0
2,14,85,2,37,10,20,aa,cc,q,999.0


In [99]:
num_df = df[['a', 'c', 'd']]
num_df

,a,c,d
0,1,4,0
1,25,12,1202
2,85,2,14


applymap, 全局的应用

In [106]:
num_p = num_df.applymap(lambda x: x + 0.2566)
num_p

,a,c,d
0,1.2566,4.2566,0.2566
1,25.2566,12.2566,1202.2566
2,85.2566,2.2566,14.2566


In [105]:
num_p.applymap(lambda x: format(x, '.2%'))

,a,c,d
0,125.66%,425.66%,25.66%
1,2525.66%,1225.66%,120225.66%
2,8525.66%,225.66%,1425.66%


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   d       3 non-null      int64 
 1   a       3 non-null      int64 
 2   c       3 non-null      int64 
 3   g       3 non-null      int64 
 4   h       3 non-null      int64 
 5   k       3 non-null      int64 
 6   i       3 non-null      object
 7   l       2 non-null      object
 8   m       3 non-null      object
dtypes: int64(6), object(3)
memory usage: 348.0+ bytes


## 注意混合内容的字符串长度的问题

In [2]:
dfa = {
    "a":[1,2],
    'b': [3,4]
}

In [3]:
dfa = pd.DataFrame(dfa)

In [4]:
# -这里创建数字的新列
dfa['c'] = [3, np.nan]

In [9]:
dfa['a'].str.len()

AttributeError: Can only use .str accessor with string values!

In [6]:
dfa['c'].str.len()

AttributeError: Can only use .str accessor with string values!

In [7]:
# --这里需要注意, 列的数据类型, 数字类型的是不支持.str的
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       2 non-null      int64  
 1   b       2 non-null      int64  
 2   c       1 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 176.0 bytes


In [10]:
# -创建混合内容
dfa['d'] = ['abc', np.nan]

In [12]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       2 non-null      int64  
 1   b       2 non-null      int64  
 2   c       1 non-null      float64
 3   d       1 non-null      object 
dtypes: float64(1), int64(2), object(1)
memory usage: 192.0+ bytes


In [13]:
dfa['d'].str.len()

0    3.0
1    NaN
Name: d, dtype: float64

注意这里的nan值不对str.len()影响, 但是其不会生成字符串的长度, 需要注意这种混合内容的字符串长度问题

当使用 `dfa['d'].str.len() == 0` 去执行某些判断时会出现问题

# 参考

- [pandas user document](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)